In [1]:
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
import os
from pyspark.ml.linalg import Vectors
from regressionLinear import regression

In [2]:
# updater tjrs les path de python car ca ne marche pas manuallement
os.environ['PYSPARK_PYTHON'] = r"C:\Users\Slash\anaconda3\envs\new_pyspark_env\python.exe"
os.environ['PYSPARK_DRIVER_PYTHON'] = r"C:\Users\Slash\anaconda3\envs\new_pyspark_env\python.exe"
# check les paths
print("PYSPARK_PYTHON:", os.environ.get('PYSPARK_PYTHON'))
print("PYSPARK_DRIVER_PYTHON:", os.environ.get('PYSPARK_DRIVER_PYTHON'))



PYSPARK_PYTHON: C:\Users\Slash\anaconda3\envs\new_pyspark_env\python.exe
PYSPARK_DRIVER_PYTHON: C:\Users\Slash\anaconda3\envs\new_pyspark_env\python.exe


## Lire la data 

In [3]:
# Initialiser la session Spark
spark = SparkSession.builder.appName("Entrainement").getOrCreate()

# Charger les données
train_df = spark.read.csv("data/train_preprocessed.csv", header=True, inferSchema=True)
test_df = spark.read.csv("data/test_preprocessed.csv", header=True, inferSchema=True)
test_df.show(3)

+------+----------+-------+-------+-------+-------+-------+--------+--------+--------+---+--------------+---------------------+---------------------+----------------------+---------------------+---------------------+---------------------+----------------------+----------------------+--------------------------+--------------------------+--------------------------+---------------------------+--------------------------+---------------------------+---------------------------+--------------------------+
|engine|cycle_time|sensor2|sensor3|sensor4|sensor7|sensor9|sensor12|sensor14|sensor17|rul|classification|sensor9_rollingmean_4|sensor3_rollingmean_4|sensor17_rollingmean_4|sensor2_rollingmean_4|sensor4_rollingmean_4|sensor7_rollingmean_4|sensor14_rollingmean_4|sensor12_rollingmean_4|sensor7_rollingmean_4_norm|sensor3_rollingmean_4_norm|sensor2_rollingmean_4_norm|sensor17_rollingmean_4_norm|sensor9_rollingmean_4_norm|sensor14_rollingmean_4_norm|sensor12_rollingmean_4_norm|sensor4_rollingmean_

In [4]:
#engines_list = train_df.select("engine").distinct().rdd.map(lambda row: row["engine"]).collect()


engines_list = [18, 30, 38, 12, 34]

In [5]:
train = train_df.select(train_df[ "sensor12_rollingmean_4_norm"],train_df["sensor7_rollingmean_4_norm"], train_df["rul"])

train_vector = train.rdd.map(lambda x: [x[2], Vectors.dense(x[0:2])]).toDF(['label','features'])

# Régression

## Régression linéaire

In [6]:
%%time
regression(train_vector,test_df,engines_list)

LR Coefficients: [151.98148729693617,91.21112385852294]
Engine #  18
average R-Squared:  0.6233644519732792
average MSE:  1786.8949427233879
average RMSE:  42.27168015023046
average MAE:  33.08518173893914
Engine #  30
average R-Squared:  0.6233644519732792
average MSE:  1786.8949427233879
average RMSE:  42.27168015023046
average MAE:  33.08518173893914
Engine #  38
average R-Squared:  0.6233644519732792
average MSE:  1786.894942723388
average RMSE:  42.27168015023046
average MAE:  33.08518173893914
Engine #  12
average R-Squared:  0.6233644519732792
average MSE:  1786.8949427233879
average RMSE:  42.27168015023046
average MAE:  33.08518173893914
Engine #  34
average R-Squared:  0.6233644519732792
average MSE:  1786.8949427233879
average RMSE:  42.27168015023046
average MAE:  33.08518173893914
CPU times: total: 375 ms
Wall time: 2min 31s


## Régression par Arbre de Décision

In [7]:
%%time
from ArbreDecision import Arbre_de_decision

Arbre_de_decision(train_vector,test_df,engines_list)

median R-Squared:  0.6577866056429873
median MSE:  1623.5838250333366
median RMSE:  40.29371942416506
median MAE:  31.409797763333252
median R-Squared:  0.6749228215291734
median MSE:  1542.2834335411624
median RMSE:  39.27191660132164
median MAE:  30.55878217728309
median R-Squared:  0.6577866056429873
median MSE:  1623.5838250333366
median RMSE:  40.29371942416506
median MAE:  31.409797763333252
median R-Squared:  0.6778772907985126
median MSE:  1528.2663652546623
median RMSE:  39.09304753091862
median MAE:  30.37980595845084
CPU times: total: 1.53 s
Wall time: 11min 33s


# Classification Binaire 

## Régression Logistique

In [4]:
from pyspark.sql.types import  DoubleType


engines_list = train_df.select("engine").distinct().rdd.map(lambda row: row["engine"]).collect()

train = train_df.select(train_df["sensor3_rollingmean_4_norm"],train_df["sensor4_rollingmean_4_norm"],
                        train_df["sensor7_rollingmean_4_norm"],
                        train_df["sensor12_rollingmean_4_norm"],
                        train_df["sensor17_rollingmean_4_norm"],train_df["classification"])

train_vector = train.rdd.map(lambda x: [x[5], Vectors.dense(x[0:5])]).toDF(['label','features'])
train_vector = train_vector.withColumn('label', train_vector['label'].cast(DoubleType()))



In [5]:
%%time

from LogisticRegression import LogsticRegression
LogsticRegression(train_vector,test_df,engines_list[:2])

Logistique Regression:
 {'Accuracy': {'mean': nan}, 'Macro Precision': {'mean': 0.824}, 'Macro Recall': {'mean': 0.832}, 'Macro F1': {'mean': 0.828}}
CPU times: total: 4.59 s
Wall time: 2min 35s


c:\Users\Slash\anaconda3\envs\new_pyspark_env\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Slash\anaconda3\envs\new_pyspark_env\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


## Arbre de decision

In [6]:
%%time

from DecisionTree_classification import DecsionTree

DecsionTree(train_vector,test_df,engines_list[:10])

Decision Tree: 
 {'Accuracy': {0.851}, 'Macro Precision': {0.85}, 'Macro Recall': {0.8299999999999998}, 'Macro F1': {0.8400000000000001}}
CPU times: total: 7.86 s
Wall time: 4min 12s


## Random Forest

In [7]:
%%time

from RandomForest import RandomForest

RandomForest(train_vector,test_df,engines_list[:10])

Random Forest: 
 {'Accuracy': {0.861}, 'Macro Precision': {0.852}, 'Macro Recall': {0.8400000000000001}, 'Macro F1': {0.8459999999999999}}
CPU times: total: 8.62 s
Wall time: 4min 32s
